In [1]:
!git clone https://github.com/vishal-burman/PyTorch-Architectures.git

Cloning into 'PyTorch-Architectures'...
remote: Enumerating objects: 367, done.
remote: Counting objects: 100% (367/367), done.
remote: Compressing objects: 100% (225/225), done.
remote: Total 798 (delta 197), reused 272 (delta 117), pack-reused 431
Receiving objects: 100% (798/798), 8.41 MiB | 11.26 MiB/s, done.
Resolving deltas: 100% (473/473), done.


In [2]:
%cd PyTorch-Architectures/modeling_gpt2/

/kaggle/working/PyTorch-Architectures/modeling_gpt2


In [4]:
! pip install transformers
! pip install datasets

     |████████████████████████████████| 147 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 17.7 MB 21.0 MB/s eta 0:00:01
     |████████████████████████████████| 243 kB 39.2 MB/s eta 0:00:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 0.16.0
    Uninstalling pyarrow-0.16.0:
      Successfully uninstalled pyarrow-0.16.0


In [5]:
import time
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from transformers import GPT2Tokenizer
from model import GPT2Classify

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dataset = load_dataset('imdb')
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model = GPT2Classify().to(device)

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3. Subsequent calls will reuse this data.


In [6]:
total_params = sum(p.numel() for p in model.parameters())
print("Total Parameters = ", total_params)

Total Parameters =  60650496


In [7]:
class TextDataset(Dataset):
    def __init__(self, tokenizer, list_texts, list_labels, max_seq_length=128):
        self.tokenizer = tokenizer
        self.list_texts = list_texts
        self.list_labels = list_labels
        self.max_seq_length = max_seq_length
        self.list_train = []
        self.build()
    
    def __len__(self):
        return len(self.list_train)
    
    def __getitem__(self, index):
        input_ids = self.list_train[index]['input_ids']
        attention_mask = self.list_train[index]['attention_mask']
        label = self.list_train[index]['label']
        return {
            'ids': torch.tensor(input_ids, dtype=torch.long),
            'mask': torch.tensor(input_ids, dtype=torch.long),
            'label': torch.tensor(label),
        }
    
    def build(self):
        for text, label in zip(self.list_texts, self.list_labels):
            tokens = self.tokenizer(text, max_length=self.max_seq_length, padding='max_length', truncation=True)
            self.list_train.append({'input_ids': tokens['input_ids'], 'attention_mask': tokens['attention_mask'], 'label': label})

In [8]:
texts_train = dataset['train']['text'][:22500]
labels_train = dataset['train']['label'][:22500]

texts_valid = dataset['train']['text'][22500:]
labels_valid = dataset['train']['label'][22500:]

In [9]:
train_dataset = TextDataset(tokenizer, texts_train, labels_train)
valid_dataset = TextDataset(tokenizer, texts_valid, labels_valid)

In [10]:
batch_size = 4
train_loader = DataLoader(dataset=train_dataset, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(dataset=valid_dataset, shuffle=False, batch_size=batch_size * 2)
test_loader = DataLoader(dataset=valid_dataset, shuffle=False, batch_size=batch_size * 2)

print("Length of Train loader: ", len(train_loader))
print("Length of Valid loader: ", len(valid_loader))
print("Length of Test loader: ", len(test_loader))

Length of Train loader:  5625
Length of Valid loader:  313
Length of Test loader:  313


In [11]:
# Check Train Loader:
for sample in train_loader:
    ids = sample['ids'].to(device)
    mask = sample['mask'].to(device)
    labels = sample['label'].to(device)
    print(ids.shape, mask.shape, labels.shape)
    break

torch.Size([4, 128]) torch.Size([4, 128]) torch.Size([4])


In [12]:
learning_rate = 5e-5
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [13]:
EPOCHS = 3

def compute_accuracy(model, data_loader, device):
    correct, total = 0, 0
    model.eval()
    with torch.set_grad_enabled(False):
        for sample in data_loader:
            ids = sample['ids']
            mask = sample['mask']
            labels = sample['labels']
            outputs = model(input_ids=ids, attention_mask=mask)
            logits = outputs[0]
            probas = F.softmax(logits, dim=-1)
            _, predicted_labels = torch.max(probas, 1)
            correct += (predicted_labels == labels).sum()
            total += labels.size(0)
    return correct.float() / total * 100
        
start_time = time.time()
for epoch in range(EPOCHS):
    model.train()
    for batch_idx, sample in enumerate(train_loader):
        ids = sample['ids'].to(device)
        mask = sample['mask'].to(device)
        labels = sample['label'].to(device)

        outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
        loss = outputs[0]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # LOGGING
        if batch_idx % 200 == 0:
            print("Batch: %04d/%04d || Epoch: %04d/%04d" % (batch_idx, len(train_loader), epoch+1, EPOCHS))
    
    model.eval()
    with torch.set_grad_enabled(False):
        # TODO define function
        train_accuracy = compute_accuracy(model, train_loader, device)
        valid_accuracy = compute_accuracy(model, valid_loader, device)
        print("Train Accuracy: %.2f || Valid Accuracy: %.2f" % (train_accuracy, valid_accuracy))
    elapsed_time = (time.time() - start_time) / 60
    print("Epoch Elapsed Time: ", elapsed_time)
elapsed_time = (time.time() - start_time) / 60
print("Total Training Time: ", elapsed_time)

RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`